In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import torch.nn as nn
import torch
import sklearn.preprocessing as preprocessing
import os

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings(action='ignore')
from copy import deepcopy

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution

In [2]:
# train.py

import os
import torch
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, log_transform
)
from strategies import BB_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions_bi_cul import inference, fitness_fn, generation_valid, generation_test, calculate_fitness

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection, ParetoLexsortSelection
from Evolution import Evolution




In [3]:
torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

prescriptor = Prescriptor(
    basic_block=None, 
    base_small_input_dim=19, 
    base_large_input_dim=19,
    base_hidden_dim=24, 
    base_output_dim=16, 
    after_input_dim=19, 
    after_hidden_dim=32, 
    after_output_dim=6, 
    num_blocks=10000,
).eval()

state_dict_path = '/root/daily/bit_2/generation/generation_5.pt'
if os.path.exists(state_dict_path):
    state_dict = torch.load(state_dict_path)
    start_gen_idx = state_dict['generation'] + 1
    best_profit = state_dict['best_profit']
    best_chromosomes = state_dict['best_chromosomes']
    # prescriptor.load_state_dict(state_dict['prescriptor_state_dict'],strict=True)

In [5]:
state_dict

{'generation': 5,
 'prescriptor_state_dict': OrderedDict([('layers.0.small_lstm.weight_ih_l0',
               tensor([[ 0.2396, -0.0374, -0.2981,  ..., -0.3690,  0.2351, -0.0326],
                       [ 0.2099, -0.1435,  0.0935,  ..., -0.0038,  0.0358, -0.1144],
                       [ 0.1518,  0.2276,  0.1299,  ..., -0.1985, -0.0857, -0.0984],
                       ...,
                       [ 0.0466, -0.2228,  0.1258,  ...,  0.1917, -0.0409, -0.4101],
                       [ 0.0398, -0.1669, -0.2139,  ..., -0.0662,  0.1150, -0.1004],
                       [ 0.0290,  0.1217, -0.0146,  ..., -0.0258,  0.0021, -0.1371]])),
              ('layers.0.small_lstm.weight_hh_l0',
               tensor([[ 0.1960, -0.0500, -0.0086,  ..., -0.1386,  0.1098, -0.0917],
                       [ 0.1960, -0.0157,  0.1827,  ..., -0.2551,  0.3336,  0.1002],
                       [-0.1907, -0.0933,  0.2206,  ...,  0.0813, -0.0943, -0.1664],
                       ...,
                       [ 0.123

In [4]:
best_profit[:, 6]

tensor([-1.0000e+09,  3.8380e-01, -1.0000e+09,  1.9379e-01])

In [7]:
optimal_solution = best_profit[(best_profit[:, 6] > 1.5) & (best_profit[:, 4] > 0.6)]
optimal_index = torch.where((best_profit[:, 6] > 1.5) & (best_profit[:, 4] > 0.6))[0]

In [8]:
optimal_index

tensor([ 1,  9, 14, 37, 49, 54, 65, 67])

In [22]:
best_profit[optimal_index][-3]

tensor([8.3728e-02, 4.0909e-02, 4.0909e-02, 1.1099e+00, 6.3295e-01, 7.7651e+02,
        8.6836e+02])

In [9]:
fitness = calculate_fitness(deepcopy(optimal_solution.numpy()))
fitness_index = np.argsort(-fitness)

In [35]:
index = 1

In [36]:
optimal_solution[fitness_index][:, 0][index]

tensor(0.8780)

In [37]:
optimal_solution[fitness_index][:, 1][index]

tensor(0.1915)

In [38]:
optimal_solution[fitness_index][:, 2][index]

tensor(0.1259)

In [39]:
optimal_solution[fitness_index][:, 3][index]

tensor(1.8843)

In [40]:
optimal_solution[fitness_index][:, 4][index]

tensor(0.8261)

In [41]:
optimal_solution[fitness_index][:, 5][index]

tensor(20.8242)

In [42]:
optimal_solution[fitness_index][:, 6][index]

tensor(1.4191)